Import all the packages

In [5]:
import stanza

AttributeError: module 'google.protobuf.internal.containers' has no attribute 'MutableMapping'

In [ ]:
import pandas as pd
import numpy as np
import re
import stanza
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import nltk

Import dataset

In [ ]:
df=pd.read_csv('garnierEnComBV.csv')

In [ ]:
#df.rename(columns={'Review':'review_text'},inplace=True)

In [ ]:
df

In [ ]:
nlp = stanza.Pipeline()

def preprocess(review):
      sentence = review
      sentence = re.sub('[^a-zA-Z]',' ',sentence)
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
      sentence = re.sub(r'\s+', ' ',sentence)

      return sentence

In [ ]:
len(df)

In [ ]:
aspect = []
for i in range(len(df)):
    txt = df['review_text'][i]
    txt=txt.lower()

    #Create a def to remove all the upper lower case, punctuations, and extra white space
    
    def preprocess(review):
      sentence = review
      sentence = re.sub('[^a-zA-Z]',' ',sentence)
      sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ',sentence)
      sentence = re.sub(r'\s+', ' ',sentence)

      return sentence

    temp=preprocess(txt)
    sentList=nltk.sent_tokenize(temp)

    #Word tokenize and Part-of-speech Tag each word

    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
    print(taggedList)

    sentList

    #Join words with two nouns together(E.g. hair product = hairproduct)then pos tag everything again

    newwordList = []
    flag = 0
    for i in range(0,len(taggedList)-1):
        if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"):
            newwordList.append(taggedList[i][0]+taggedList[i+1][0])
            flag=1
        else:
            if(flag==1):
                flag=0
                continue
            newwordList.append(taggedList[i][0])
            if(i==len(taggedList)-2):
                newwordList.append(taggedList[i+1][0])
    finaltxt = ' '.join(word for word in newwordList)
    print('final',finaltxt)

    sentList = nltk.sent_tokenize(finaltxt)
    for line in sentList:
        txt_list = nltk.word_tokenize(line)
        taggedList = nltk.pos_tag(txt_list)
    print(taggedList)

    taggedList

    stop_words = set(stopwords.words('english'))
    new_txt_list = nltk.word_tokenize(finaltxt)
    wordsList = [w for w in new_txt_list if not w in stop_words]
    taggedList = nltk.pos_tag(wordsList)
    
    #exception for GW
    for i in range(0,len(taggedList)):
        if taggedList[i][1]=='GW':
            new=(taggedList[i][0],'JJ')
            taggedList[i] = new

    doc = nlp(finaltxt)
    dep_node = []
    for dep_edge in doc.sentences[0].dependencies:
        dep_node.append([dep_edge[2].text, dep_edge[0].id, dep_edge[1]])
    for i in range(0, len(dep_node)):
        if (int(dep_node[i][1]) != 0):
            dep_node[i][1] = newwordList[(int(dep_node[i][1]) - 1)]
    

    (doc.sentences[0].dependencies)[1]

    featureList = []
    categories = []
    totalfeatureList = []
    print(taggedList)
    for i in taggedList:
        if(i[1]=='JJ' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
            featureList.append(list(i))
            totalfeatureList.append(list(i)) # This list will store all the features for every sentence
            categories.append(i[0])
    #print(featureList)
    #print(totalfeatureList)
    #print(categories)

    fcluster = []
    for i in featureList:
        filist = []
        for j in dep_node:
            if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                if(j[0]==i[0]):
                    filist.append(j[1])
                else:
                    filist.append(j[0])
        fcluster.append([i[0], filist])
    

    finalcluster = []
    dic = {}
    for i in featureList:
        dic[i[0]] = i[1]
    for i in fcluster:
        if(dic[i[0]]=="NN" or dic[i[0]]=="NNS" or dic[i[0]]=="NNP" or dic[i[0]]=="NNPS"):
            finalcluster.append(i)
    print(finalcluster)
    aspect.append(finalcluster)
df['cluster']=aspect
    

In [ ]:
featureList

In [ ]:
df['review_text'][2]